In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization, LeakyReLU,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
dataset_path = "Data\\SOCOFing\\Real"
img_height, img_width = 96, 96

data_paths = [
    "Data\\SOCOFing\\Real",
    "Data\\SOCOFing\\Altered\\Altered-Easy",
    "Data\\SOCOFing\\Altered\\Altered-Medium",
    "Data\SOCOFing\Altered\Altered-Hard"  
]
def preprocess_images(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img_width, img_height))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    return img


def load_dataset(data_paths):
    images = []
    labels = []
    for path in data_paths:
        for file in os.listdir(path):
            if file.endswith(".BMP"):
                img_path = os.path.join(path, file)
                img = preprocess_images(img_path)
                images.append(img)
                label = int(file.split("__")[0]) - 1
                labels.append(label)
    return np.array(images), np.array(labels)

images, labels = load_dataset(data_paths)


label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [ ]:
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), input_shape=(img_height, img_width, 1)),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3)),
        MaxPooling2D((2, 2)),
        
        Conv2D(128, (3, 3)),
        MaxPooling2D((2, 2)),
        
        Conv2D(256, (3, 3)),
        MaxPooling2D((2, 2)),
        
        GlobalAveragePooling2D(),
        
        Dense(512, activation='relu'),
        Dropout(0.5),
        
        Dense(256, activation='relu'),
        Dropout(0.5),
        
        Dense(600, activation='softmax')  # Lớp đầu ra cho phân loại
    ])
    return model
model = create_model()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
checkpoint = ModelCheckpoint('model.keras', save_best_only=True, monitor='val_loss', mode="min")
early_stopping = EarlyStopping(monitor = 'val_accuracy', patience = 15, mode="max",restore_best_weights=True)